In [2]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

# Set the seed value for experiment reproducibility.
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [3]:
# shuffle filenames

data_dir = "recordings"

filenames = tf.io.gfile.glob(str(data_dir) + "/*")
filenames = tf.random.shuffle(filenames)
num_samples = len(filenames)

print(num_samples)
print(filenames[:5])

3000
tf.Tensor(
[b'recordings/2_lucas_36.wav' b'recordings/8_nicolas_49.wav'
 b'recordings/4_jackson_21.wav' b'recordings/6_george_29.wav'
 b'recordings/4_nicolas_14.wav'], shape=(5,), dtype=string)


2022-04-14 23:34:04.463874: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
tenth = int(num_samples * 0.1)
eightyth = tenth * 8

train_files = filenames[:eightyth]
val_files = filenames[eightyth: eightyth + tenth]
test_files = filenames[eightyth + tenth:]

print('Training set size', len(train_files))
print('Validation set size', len(val_files))
print('Test set size', len(test_files))

Training set size 2400
Validation set size 300
Test set size 300


In [15]:
def decode_audio(audio_binary):
    # Decode WAV-encoded audio files to `float32` tensors, normalized
    # to the [-1.0, 1.0] range. Return `float32` audio and a sample rate.
    audio, _ = tf.audio.decode_wav(contents=audio_binary)
    # Since all the data is single channel (mono), drop the `channels`
    # axis from the array.
    return tf.squeeze(audio, axis=-1)

def get_label(file_path):
    parts = tf.strings.split(
      input=file_path,
      sep=os.path.sep)
    
    filename = parts[1].numpy().decode("utf-8")
    label = int(filename.split("_")[0])
    
    # Note: You'll use indexing here instead of tuple unpacking to enable this
    # to work in a TensorFlow graph.
    return label

def get_waveform_and_label(file_path):
    label = get_label(file_path)
    audio_binary = tf.io.read_file(file_path)
    waveform = decode_audio(audio_binary)
    return waveform, label

In [17]:
tf.config.run_functions_eagerly(True)

AUTOTUNE = tf.data.AUTOTUNE

files_ds = tf.data.Dataset.from_tensor_slices(train_files)

# i = 0
# for element in files_ds:
#     print(element)
#     print(get_waveform_and_label(element))
#     i += 1
#     if i > 10:
#         break


waveform_ds = files_ds.map(
    map_func=get_waveform_and_label)

AttributeError: in user code:

    File "/var/folders/vs/b_y62_cj4859tj4h1ch0glg80000gn/T/ipykernel_3306/147808418.py", line 22, in get_waveform_and_label  *
        label = get_label(file_path)
    File "/var/folders/vs/b_y62_cj4859tj4h1ch0glg80000gn/T/ipykernel_3306/147808418.py", line 14, in get_label  *
        filename = parts[1].numpy().decode("utf-8")

    AttributeError: 'Tensor' object has no attribute 'numpy'


In [19]:
tf.executing_eagerly()

True